In [27]:
import os
import cx_Oracle
import pandas as pd
import numpy as np
import pickle
from datetime import date,datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV, cross_val_score 
import time
import matplotlib.pyplot as plt
from sklearn.utils import resample
import datetime

In [29]:
os.chdir(r"C:\Users\aman.khatri91425\Desktop\HomeCredit\Bigdata\Common Maps KNN")

In [30]:
df= pd.read_csv(os.getcwd()+r"\Data Files\prod_skp_scored.csv")

In [43]:
df.columns

Index(['Unnamed: 0', 'SKP_CREDIT_CASE', 'DATE_DECISION', 'CUS_LAT', 'CUS_LONG',
       'GRID_ID', 'RISK_FPD10', 'RISK_FPD30', 'RISK_FSTQPD30',
       'CLIENT_INR_INCOME', 'PRODUCT_GROUP', 'NTC_FLAG', 'RISK_FPD10_SCORE',
       'G_RISK_FPD10_SCORE', 'RISK_FPD30_SCORE', 'G_RISK_FPD30_SCORE',
       'RISK_FSTQPD30_SCORE', 'G_RISK_FSTQPD30_SCORE', 'RISK_FPD10_MM_SCORE',
       'G_RISK_FPD10_MM_SCORE', 'DEFAULT_FLAG_CHANGE_1',
       'G_DEFAULT_FLAG_CHANGE_1', 'G_NO_PAYMENT_CURRENT_PAYMENT_1',
       'G_NO_PAYMENT_CURRENT_PAYMENT_2', 'NO_PAYMENT_CURRENT_PAYMENT_1',
       'NO_PAYMENT_CURRENT_PAYMENT_2', 'LOGIT_G_RISK_FPD10_SCORE',
       'LOGIT_G_RISK_FPD30_SCORE', 'LOGIT_G_RISK_FSTQPD30_SCORE',
       'LOGIT_G_RISK_FPD10_MM_SCORE', 'LOGIT_G_DEFAULT_FLAG_CHANGE_1',
       'LOGIT_G_NO_PAYMENT_CURRENT_PAYMENT_1',
       'LOGIT_G_NO_PAYMENT_CURRENT_PAYMENT_2', 'COMBINED_SCORE_KNN', 'MONTH'],
      dtype='object')

In [34]:
df["DATE_DECISION"] = pd.to_datetime(df.DATE_DECISION,format = '%Y-%m-%d')

In [35]:
TIME_DECISION_DATE = "DATE_DECISION"
FINAL_SCORE = "COMBINED_SCORE_KNN"
PREDICTOR_LIST = ['G_RISK_FPD10_SCORE','G_RISK_FPD30_SCORE','G_RISK_FSTQPD30_SCORE','G_RISK_FPD10_MM_SCORE','G_DEFAULT_FLAG_CHANGE_1',
                 'G_NO_PAYMENT_CURRENT_PAYMENT_1','G_NO_PAYMENT_CURRENT_PAYMENT_2']
df["MONTH"] = df[TIME_DECISION_DATE].dt.strftime("%y-%m")

In [44]:
TARGET = {
          "RISK_FSTQPD30":150,
#           "RISK_FSTPD30":120,
#           "RISK_FSPD30":90,
          "RISK_FPD30":60
         }


In [49]:
from sklearn.metrics import roc_auc_score
gini_dic = {}
for x in df["MONTH"].unique():
    for y in TARGET:        
        if "MONTH" not in gini_dic:
            gini_dic["MONTH"] = []
        if "TARGET" not in gini_dic:
            gini_dic["TARGET"] = []
        if FINAL_SCORE not in gini_dic: 
            gini_dic[FINAL_SCORE] = []
        gini_dic["MONTH"].append(x)
        gini_dic["TARGET"].append(y)
        if df[(df["MONTH"]==x)&(df["DATE_DECISION"]<= datetime.datetime(2020,4,15)-datetime.timedelta(TARGET[y]))].shape[0]>0:
            gini_dic[FINAL_SCORE].append((2*roc_auc_score(df[df["MONTH"]==x][y],df[df["MONTH"]==x][FINAL_SCORE])-1)*100)
            for z in PREDICTOR_LIST:
                if z not in gini_dic:
                      gini_dic[z] = []
                gini_dic[z].append(((2*roc_auc_score(df[df["MONTH"]==x][y],df[df["MONTH"]==x][z])-1)*100))
        else:
            gini_dic[FINAL_SCORE].append(np.nan)
            for z in PREDICTOR_LIST:
                if z not in gini_dic:
                      gini_dic[z] = []
                gini_dic[z].append(np.nan)
    
gini_df = pd.DataFrame(gini_dic)
gini_df

,MONTH,TARGET,COMBINED_SCORE_KNN,G_RISK_FPD10_SCORE,G_RISK_FPD30_SCORE,G_RISK_FSTQPD30_SCORE,G_RISK_FPD10_MM_SCORE,G_DEFAULT_FLAG_CHANGE_1,G_NO_PAYMENT_CURRENT_PAYMENT_1,G_NO_PAYMENT_CURRENT_PAYMENT_2
0,19-01,RISK_FSTQPD30,16.595819,9.919072,7.077606,10.786177,8.951594,3.265257,3.840878,2.616260
1,19-01,RISK_FPD30,16.634391,13.164055,13.893239,10.388006,12.326014,-2.399949,5.957648,5.435619
2,19-02,RISK_FSTQPD30,17.216578,9.488639,7.440386,10.464644,8.542652,4.478985,3.167252,3.218154
3,19-02,RISK_FPD30,19.792724,16.106456,16.654474,11.878243,16.365349,1.445177,3.452799,3.646315
4,19-03,RISK_FSTQPD30,17.073465,8.367878,6.326416,10.317785,7.601651,4.504851,2.010351,1.846512
5,19-03,RISK_FPD30,18.788577,15.461588,15.729513,12.783112,16.403551,0.008898,4.522400,4.216954
6,19-04,RISK_FSTQPD30,16.623069,7.318627,5.712028,10.344717,6.348350,4.609051,3.214185,1.519635
7,19-04,RISK_FPD30,17.220452,14.858553,14.778821,12.887499,14.337828,-0.387855,7.279039,4.781991
8,19-05,RISK_FSTQPD30,15.644055,6.756231,5.646873,9.483772,6.373400,4.906788,1.733231,2.197567
9,19-05,RISK_FPD30,16.980732,12.446672,14.274095,11.361183,11.623042,0.372844,7.248830,7.063335


In [50]:
gini_df.to_csv(os.getcwd()+"\gini_short_target.csv")

In [52]:
df=df[df["NTC_FLAG"]=="NTC"]

In [54]:
from sklearn.metrics import roc_auc_score
gini_dic = {}
for x in df["MONTH"].unique():
    for y in TARGET:        
        if "MONTH" not in gini_dic:
            gini_dic["MONTH"] = []
        if "TARGET" not in gini_dic:
            gini_dic["TARGET"] = []
        if FINAL_SCORE not in gini_dic: 
            gini_dic[FINAL_SCORE] = []
        gini_dic["MONTH"].append(x)
        gini_dic["TARGET"].append(y)
        if df[(df["MONTH"]==x)&(df["DATE_DECISION"]<= datetime.datetime(2020,4,15)-datetime.timedelta(TARGET[y]))].shape[0]>0:
            gini_dic[FINAL_SCORE].append((2*roc_auc_score(df[df["MONTH"]==x][y],df[df["MONTH"]==x][FINAL_SCORE])-1)*100)
            for z in PREDICTOR_LIST:
                if z not in gini_dic:
                      gini_dic[z] = []
                gini_dic[z].append(((2*roc_auc_score(df[df["MONTH"]==x][y],df[df["MONTH"]==x][z])-1)*100))
        else:
            gini_dic[FINAL_SCORE].append(np.nan)
            for z in PREDICTOR_LIST:
                if z not in gini_dic:
                      gini_dic[z] = []
                gini_dic[z].append(np.nan)
    
gini_df = pd.DataFrame(gini_dic)
gini_df

,MONTH,TARGET,COMBINED_SCORE_KNN,G_RISK_FPD10_SCORE,G_RISK_FPD30_SCORE,G_RISK_FSTQPD30_SCORE,G_RISK_FPD10_MM_SCORE,G_DEFAULT_FLAG_CHANGE_1,G_NO_PAYMENT_CURRENT_PAYMENT_1,G_NO_PAYMENT_CURRENT_PAYMENT_2
0,19-01,RISK_FSTQPD30,11.052674,9.760295,7.235715,10.595149,8.726585,1.760610,4.267934,3.309712
1,19-01,RISK_FPD30,10.493013,13.151922,14.767014,10.191785,11.853774,-4.163997,6.397983,6.100035
2,19-02,RISK_FSTQPD30,11.108417,9.310611,7.929943,10.313403,8.286928,2.283609,4.574704,4.444959
3,19-02,RISK_FPD30,13.188370,15.872502,17.348023,11.876989,16.576317,-1.386202,5.116743,5.629404
4,19-03,RISK_FSTQPD30,10.801311,8.376182,6.791345,10.604359,7.523871,2.930474,3.139865,2.973849
5,19-03,RISK_FPD30,14.604433,16.288486,17.620689,13.503548,17.201528,-2.404326,6.442476,5.467057
6,19-04,RISK_FSTQPD30,10.355705,7.128740,5.757637,10.046726,5.959732,3.408314,4.004166,2.446486
7,19-04,RISK_FPD30,13.777171,15.052725,16.286212,12.366354,14.663937,-2.909229,8.723845,6.186486
8,19-05,RISK_FSTQPD30,10.662431,7.338691,6.248995,10.480952,6.319742,3.347694,2.950858,3.684568
9,19-05,RISK_FPD30,12.116619,12.797866,15.439576,11.076217,10.543350,-1.299303,8.127265,7.908592


In [55]:
gini_df.to_csv(os.getcwd()+"\gini_short_target_ntc.csv")

In [56]:
df[["MONTH","RISK_FPD30"]].groupby("MONTH").agg(['count','sum'])

RISK_FPD30        
           count     sum
MONTH                   
19-01     206415  2887.0
19-02     180959  2820.0
19-03     194449  3266.0
19-04     178069  2675.0
19-05     167244  2302.0
19-06     163907  2025.0
19-07     151180  1841.0
19-08     163921  2110.0
19-09     159536  2421.0
19-10     259849  3229.0
19-11     176158  2282.0
19-12     168345  2261.0
20-01     180297  1875.0
20-02     157668   179.0
20-03      98660     0.0